# Imports

In [1]:
import numpy as np
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import make_scorer, fbeta_score

from skopt.space import Integer, Categorical

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import fbeta_score, roc_auc_score, f1_score
from sklearn.metrics import accuracy_score, precision_score, recall_score
import shap

from skopt import BayesSearchCV

from typing import Dict, List, Literal
from skopt.space import Real, Integer, Categorical

from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler, SMOTE
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
import warnings
warnings.filterwarnings("ignore")

## Read dataset

In [2]:
data = pd.read_csv("../data/processed/20. FINAL_mean_delta_multi_output.csv")
print(data.shape)
data.head()

(6091, 210)


,Hba1c,Hba1c Time,Hba1c FM,Hba1c FM Time,BMI,BMI Time,Cancer,Cancer Time,Carotid Disease,Carotid Disease Time,...,y_stk_or_aemb_3_months,y_stk_or_aemb_6_months,y_stk_or_aemb_12_months,y_stk_or_aemb_24_months,y_stk_or_aemb,History of Vascular Disease,Antihypertensive Medication,Diabetes Mellitus,Diabetes Medication,Abnormal Kidney Function
0,6.26,-501,6.26,-501,20.7,-19,0,10000,0,10000,...,0,0,0,0,0,0,1,0,0,0
1,6.26,-501,6.26,-501,26.7,-780,1,-4846,0,10000,...,0,0,0,0,0,0,1,0,0,0
2,5.80,-287,6.30,-2701,31.1,-35,0,10000,0,10000,...,0,0,0,0,0,0,1,1,1,1
3,6.26,-501,6.26,-501,21.3,-207,0,10000,0,10000,...,0,0,0,0,0,1,1,0,0,1
4,5.90,-162,5.40,-5209,37.8,-554,1,-86,0,10000,...,0,0,0,0,0,1,1,1,1,0


## Drop columns

In [3]:
targets = ["y_acs_6_months", "y_cvdeath_6_months", "y_death_6_months", "y_hf_6_months", "y_inp_6_months", "y_stk_or_aemb"]

# keep only target Ys, drop any other y_* cols first
cols_to_drop = [col for col in data.columns if col.startswith('y_') and col not in targets]
data_all_features = data.drop(columns=cols_to_drop)

# start longitudinal from this cleaned version
data_slope = data_all_features.copy()

# drop *_t
cols_to_drop = [col for col in data_slope.columns if col.endswith('Time')]
data_slope = data_slope.drop(columns=cols_to_drop)

# drop FM
cols_to_drop = [col for col in data_slope.columns if "FM" in col]
data_slope = data_slope.drop(columns=cols_to_drop)

# static = longitudinal without delta
data_static = data_slope.copy()

cols_to_drop = [col for col in data_slope.columns if col.startswith('Δ')]
data_static = data_static.drop(columns=cols_to_drop)


In [4]:
data_static.head()

,Hba1c,BMI,Cancer,Carotid Disease,Coronary Disease,COPD,Creatinine,DBP,Dyslipidemia,eGFR,...,y_cvdeath_6_months,y_death_6_months,y_hf_6_months,y_inp_6_months,y_stk_or_aemb,History of Vascular Disease,Antihypertensive Medication,Diabetes Mellitus,Diabetes Medication,Abnormal Kidney Function
0,6.26,20.7,0,0,0,0,0.99,87.0,1,64.15,...,0,0,0,0,0,0,1,0,0,0
1,6.26,26.7,1,0,0,0,0.53,81.0,0,88.50,...,0,0,1,1,0,0,1,0,0,0
2,5.80,31.1,0,0,0,0,0.88,109.0,1,46.80,...,0,0,0,0,0,0,1,1,1,1
3,6.26,21.3,0,0,0,0,1.56,63.0,1,9.00,...,0,0,0,0,0,1,1,0,0,1
4,5.90,37.8,1,0,1,0,0.64,98.0,1,76.60,...,0,0,1,1,0,1,1,1,1,0


## Models settings

In [5]:
# Define 5-fold stratified CV
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Create F2 scorer
f2_scorer = make_scorer(fbeta_score, beta=2)

# ===================== Naive Bayes =====================
search_space_nb = {
    "var_smoothing": (1e-12, 1e-1, "log-uniform")
}

nb_opt = BayesSearchCV(
    estimator=GaussianNB(),
    search_spaces=search_space_nb,
    scoring=f2_scorer,
    n_iter=30,
    cv=cv,
    n_jobs=-1,
    random_state=42
)

# ===================== Logistic Regression =====================
search_space_lr = {
    'C': (1e-4, 1e+3, 'log-uniform'),
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear']
}

lr_opt = BayesSearchCV(
    LogisticRegression(max_iter=1000, random_state=42),
    search_spaces=search_space_lr,
    scoring=f2_scorer,
    n_iter=30,
    cv=cv,
    n_jobs=-1,
    random_state=42
)

# ===================== Decision Tree =====================
search_space_dt = {
    "max_depth": Integer(3, 20),
    "min_samples_split": Integer(2, 20),
    "min_samples_leaf": Integer(1, 10),
    "criterion": Categorical(["gini", "entropy"])
}

dt_opt = BayesSearchCV(
    DecisionTreeClassifier(random_state=42),
    search_spaces=search_space_dt,
    n_iter=30,
    scoring=f2_scorer,
    cv=cv,
    n_jobs=-1,
    random_state=42,
    verbose=0
)

# ===================== Random Forest =====================
search_space_rf = {
    'n_estimators': (50, 300),
    'max_depth': (1, 50),
    'min_samples_split': (2, 20),
    'min_samples_leaf': (1, 20),
    'max_features': ['sqrt', 'log2', None],
    'bootstrap': [True, False]
}

rf_opt = BayesSearchCV(
    RandomForestClassifier(random_state=42),
    search_spaces=search_space_rf,
    scoring=f2_scorer,
    n_iter=30,
    cv=cv,
    n_jobs=-1,
    random_state=42
)

# ===================== XGBoost =====================
search_space_xgb = {
    'n_estimators': (10, 500),
    'max_depth': (1, 50),
    'learning_rate': (0.001, 0.1, 'uniform'),
    'subsample': (0.5, 1.0),
    'colsample_bytree': (0.5, 1.0),
    'gamma': (0, 5),
    'min_child_weight': (1, 20)
}

xgb_opt = BayesSearchCV(
    XGBClassifier(random_state=42),
    search_spaces=search_space_xgb,
    scoring=f2_scorer,
    n_iter=30,
    n_jobs=-1,
    random_state=42
)

# ===================== MLP ===============================

mlp_opt = MLPClassifier(max_iter=2000, solver="adam", activation="relu", hidden_layer_sizes=(200,100), random_state=42)

# ===================== Optimizer Map =====================
optimizer_map = {
    "nb": nb_opt,
    "lr": lr_opt,
    "dt": dt_opt,
    "rf": rf_opt,
    "xgb": xgb_opt,
    "mlp": mlp_opt
}


## Define function

In [6]:
SamplingName = Literal["baseline", "undersample", "oversample", "smote", "all"]
ModelName = Literal["nb", "lr", "xgb", "dt", "rf", "mlp"]
ExplainerName = Literal["none", "linear", "tree", "kernel"]

def _make_resampler(name: SamplingName, random_state: int, y_train: pd.Series = None):
    if name == "baseline":
        return None
    if name == "undersample":
        if y_train is None:
            raise ValueError("y_train must be provided for undersampling strategy")
        n_minority = y_train.sum()
        n_required = max(int(0.1 * len(y_train)), n_minority * 2)
        n_majority = n_required - n_minority
        n_majority = min(n_majority, (y_train == 0).sum())
        sampling_strategy = {0: n_majority, 1: n_minority}
        return RandomUnderSampler(sampling_strategy=sampling_strategy, random_state=random_state)
    if name == "oversample":
        return RandomOverSampler(random_state=random_state)
    if name == "smote":
        return SMOTE(random_state=random_state)
    raise ValueError(f"Unknown sampling technique: {name}")

def run_cv_with_sampling(
    X_full: pd.DataFrame,
    target_cols: List[str],
    target_name: str,
    optimizer_map: Dict[ModelName, object],
    model_name: ModelName = "nb",
    sampling: SamplingName = "all",
    n_splits: int = 5,
    random_state: int = 42,
    xgb_04: bool = False,
    explainer: ExplainerName = "none",
    max_disp = 20,
) -> Dict[str, dict]:
    if target_name not in target_cols:
        raise ValueError("target_name must be inside target_cols")
    missing_targets = [c for c in target_cols if c not in X_full.columns]
    if missing_targets:
        raise ValueError(f"Target columns not in X_full: {missing_targets}")

    y = X_full[target_name].copy()
    X = X_full.drop(columns=target_cols).copy()

    cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)

    if sampling == "all":
        techniques = ["baseline", "undersample", "oversample", "smote"]
    else:
        techniques = [sampling]

    results: Dict[str, dict] = {}
    fold_results: List[dict] = []   # collect across folds

    for tech in techniques:
        print(f"\n=== Technique: {tech.upper()} ===")

        fold_results = []
        best_params_per_fold: List[dict] = []

        for fold, (train_idx, val_idx) in enumerate(cv.split(X, y), start=1):
            X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
            y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

            if model_name in ["nb", "lr", "mlp"]:
                scaler = StandardScaler()
                X_train = scaler.fit_transform(X_train)
                X_val = scaler.transform(X_val)

            resampler = _make_resampler(tech, random_state, y_train)
            if resampler is not None:
                X_train_rs, y_train_rs = resampler.fit_resample(X_train, y_train)
            else:
                X_train_rs, y_train_rs = X_train, y_train

            opt = optimizer_map[model_name]
            opt.fit(X_train_rs, y_train_rs)

            if model_name == "mlp":
                best_model = opt
                best_params_per_fold = {}
            else:
                best_model = opt.best_estimator_
                best_params_per_fold.append(getattr(opt, "best_params_", {}))

            y_pred = best_model.predict(X_val)

            if hasattr(best_model, "predict_proba"):
                y_pred_proba = best_model.predict_proba(X_val)[:, 1]
            else:
                scores = best_model.decision_function(X_val)
                smin, smax = scores.min(), scores.max()
                y_pred_proba = (scores - smin) / (smax - smin + 1e-9)

            if model_name == "xgb" and xgb_04:
                y_pred = (y_pred_proba >= 0.4).astype(int)

            prec = precision_score(y_val, y_pred, zero_division=0)
            rec = recall_score(y_val, y_pred, zero_division=0)
            f1 = f1_score(y_val, y_pred, zero_division=0)
            fbeta2 = fbeta_score(y_val, y_pred, beta=2, zero_division=0)
            roc = roc_auc_score(y_val, y_pred_proba)

            fold_metrics = {
                "fold": fold,
                "accuracy": accuracy_score(y_val, y_pred),
                "precision": prec,
                "sensitivity": rec,
                "f1_score": f1,
                "fbeta_2": fbeta2,
                "roc_auc": roc,
                "NNS": (1 / prec) if prec > 0 else np.inf,
                "best_params": getattr(opt, "best_params_", {}),
            }
            fold_results.append(fold_metrics)

            print({k: (round(v, 3) if isinstance(v, float) else v) for k, v in fold_metrics.items()})

        metrics = [
            "accuracy",
            "precision",
            "sensitivity",
            "f1_score",
            "fbeta_2",
            "roc_auc",
            "NNS",
        ]
        mean_std = {
            m: (np.mean([fr[m] for fr in fold_results]), np.std([fr[m] for fr in fold_results]))
            for m in metrics
        }

        print("\nMean scores across folds (", tech, "):")
        for m in metrics:
            mu, sd = mean_std[m]
            print(f"{m}: {mu:.3f} \u00B1 {sd:.3f}")

        summary_df = pd.DataFrame(fold_results)
        results[tech] = {
            "fold_results": fold_results,
            "summary_metrics": summary_df,
            "best_params_per_fold": best_params_per_fold,
        }

    # ---- FINAL MODEL FIT AND SHAP ----
    best_params_overall = max(fold_results, key=lambda x: x["f1_score"])["best_params"]

    # rebuild the chosen model with best params
    if model_name == "rf":
        final_model = RandomForestClassifier(**best_params_overall)
    elif model_name == "xgb":
        final_model = XGBClassifier(**best_params_overall, use_label_encoder=False, eval_metric="logloss")
    elif model_name == "dt":
        final_model = DecisionTreeClassifier(**best_params_overall)
    elif model_name == "lr":
        final_model = LogisticRegression(**best_params_overall, max_iter=1000)
    elif model_name == "nb":
        final_model = GaussianNB(**best_params_overall)
    elif model_name == "mlp":
        final_model = MLPClassifier(**best_params_overall, max_iter=2000)
    else:
        raise ValueError(f"Unknown model_name: {model_name}")

    final_model.fit(X, y)

    if explainer != "none":
        X_background_final = X.sample(min(100, len(X)), random_state=random_state)
        X_sample_final = X.sample(min(200, len(X)), random_state=random_state)

        if explainer == "kernel":
            predict_fn_final = lambda x: final_model.predict_proba(x)[:, 1]
            shap_explainer = shap.KernelExplainer(predict_fn_final, X_background_final)
            shap_values_final = shap_explainer.shap_values(X_sample_final)
        elif explainer == "tree":
            shap_explainer = shap.TreeExplainer(final_model)
            shap_values_final = shap_explainer.shap_values(X_sample_final)
        elif explainer == "linear":
            shap_explainer = shap.LinearExplainer(final_model, X_background_final)
            shap_values_final = shap_explainer.shap_values(X_sample_final)

        shap.summary_plot(
            shap_values_final,
            X_sample_final,
            feature_names=X.columns,
            max_display=max_disp
        )

    return results

## Stroke Static LR

In [ ]:
targets = [
    "y_stk_or_aemb_1_month", "y_stk_or_aemb_3_months", "y_stk_or_aemb_6_months", 
    "y_stk_or_aemb_12_months", "y_stk_or_aemb_24_months", "y_stk_or_aemb",

    "y_death_1_month", "y_death_3_months", "y_death_6_months", 
    "y_death_12_months", "y_death_24_months", "y_death",

    "y_cvdeath_1_month", "y_cvdeath_3_months", "y_cvdeath_6_months", 
    "y_cvdeath_12_months", "y_cvdeath_24_months", "y_cvdeath",

    "y_hf_1_month", "y_hf_3_months", "y_hf_6_months", 
    "y_hf_12_months", "y_hf_24_months", "y_hf",

    "y_inp_1_month", "y_inp_3_months", "y_inp_6_months", 
    "y_inp_12_months", "y_inp_24_months", "y_inp",

    "y_acs_1_month", "y_acs_3_months", "y_acs_6_months", 
    "y_acs_12_months", "y_acs_24_months", "y_acs"
]

# keep only target Ys, drop any other y_* cols first
cols_to_drop = [col for col in data.columns if col.startswith('y_') and col not in targets]
data_all_features = data.drop(columns=cols_to_drop)

# start longitudinal from this cleaned version
data_slope = data_all_features.copy()

# drop *_t
cols_to_drop = [col for col in data_slope.columns if col.endswith('Time')]
data_slope = data_slope.drop(columns=cols_to_drop)

# drop FM
cols_to_drop = [col for col in data_slope.columns if "FM" in col]
data_slope = data_slope.drop(columns=cols_to_drop)

# static = longitudinal without delta
data_static = data_slope.copy()

cols_to_drop = [col for col in data_slope.columns if col.startswith('Δ')]
data_static = data_static.drop(columns=cols_to_drop)

In [8]:
results = run_cv_with_sampling(
    X_full=data_static,
    target_cols=targets,
    target_name="y_stk_or_aemb_24_months",
    optimizer_map=optimizer_map,
    model_name="lr",
    sampling="smote",
    n_splits=5,
    random_state=42,
    explainer="none"
)


=== Technique: SMOTE ===
{'fold': 1, 'accuracy': 0.572, 'precision': 0.019, 'sensitivity': 0.625, 'f1_score': 0.037, 'fbeta_2': 0.085, 'roc_auc': np.float64(0.606), 'NNS': 52.6, 'best_params': OrderedDict([('C', 0.00025867031730454013), ('penalty', 'l2'), ('solver', 'liblinear')])}
{'fold': 2, 'accuracy': 0.588, 'precision': 0.016, 'sensitivity': 0.5, 'f1_score': 0.031, 'fbeta_2': 0.071, 'roc_auc': np.float64(0.537), 'NNS': 62.75, 'best_params': OrderedDict([('C', 0.0004048464889463556), ('penalty', 'l2'), ('solver', 'liblinear')])}
{'fold': 3, 'accuracy': 0.568, 'precision': 0.017, 'sensitivity': 0.562, 'f1_score': 0.033, 'fbeta_2': 0.076, 'roc_auc': np.float64(0.525), 'NNS': 58.667, 'best_params': OrderedDict([('C', 0.0001566482327075753), ('penalty', 'l2'), ('solver', 'liblinear')])}
{'fold': 4, 'accuracy': 0.61, 'precision': 0.023, 'sensitivity': 0.688, 'f1_score': 0.044, 'fbeta_2': 0.101, 'roc_auc': np.float64(0.667), 'NNS': 43.727, 'best_params': OrderedDict([('C', 0.00445180964

In [9]:
results = run_cv_with_sampling(
    X_full=data_static,
    target_cols=targets,
    target_name="y_stk_or_aemb_12_months",
    optimizer_map=optimizer_map,
    model_name="lr",
    sampling="smote",
    n_splits=5,
    random_state=42,
    explainer="none"
)


=== Technique: SMOTE ===
{'fold': 1, 'accuracy': 0.797, 'precision': 0.016, 'sensitivity': 0.333, 'f1_score': 0.031, 'fbeta_2': 0.068, 'roc_auc': np.float64(0.617), 'NNS': 61.0, 'best_params': OrderedDict([('C', 487.9169484427551), ('penalty', 'l2'), ('solver', 'liblinear')])}
{'fold': 2, 'accuracy': 0.698, 'precision': 0.008, 'sensitivity': 0.273, 'f1_score': 0.016, 'fbeta_2': 0.037, 'roc_auc': np.float64(0.552), 'NNS': 121.0, 'best_params': OrderedDict([('C', 0.00277290682409599), ('penalty', 'l2'), ('solver', 'liblinear')])}
{'fold': 3, 'accuracy': 0.548, 'precision': 0.016, 'sensitivity': 0.818, 'f1_score': 0.032, 'fbeta_2': 0.075, 'roc_auc': np.float64(0.762), 'NNS': 62.0, 'best_params': OrderedDict([('C', 0.00014419146277986448), ('penalty', 'l2'), ('solver', 'liblinear')])}
{'fold': 4, 'accuracy': 0.663, 'precision': 0.017, 'sensitivity': 0.636, 'f1_score': 0.033, 'fbeta_2': 0.076, 'roc_auc': np.float64(0.665), 'NNS': 59.143, 'best_params': OrderedDict([('C', 0.0011081095784807

## All cause death slope based XGB

## cardiovascular death staticc XGB

## HF hospitalization Static XGB

## Inpatient Visit Static LR

## ACS ALl-features MLP